# Label each row with time remaining until end of each segment

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import datetime as dt
import time
from datetime import datetime

In [ ]:
df = pds.read_csv('../../bus203_all.csv')
df.head()

Ignore entries that are not `ObservedPositionEvent`

In [ ]:
df = df[df['event'] == "ObservedPositionEvent"]

This leaves the indexes of rows untouched, reset.

In [ ]:
df = df.reset_index().drop(columns=['index'])

A triple loop as it seems, but it is only to finally group rows from each individual segment from every journey. Should be linear in time, as the innermost loop will do all executions and every row is visited once. This only took ~50% of my 8GB of RAM but took 850 seconds to run.

In [ ]:
ts = pds.DataFrame(np.zeros(len(df.index)), columns=list('t'))

t0 = time.time()

for j, df_j in df.groupby('journey_number'):
    for k, df_s in df_j.groupby('segment_number'):
        end_time = df_s['timestamp'].iloc[-1]
        for idx, row in df_s.iterrows():
            # The subtraction returns timedelta between the two timestamp objects
            # and total seconds convert the pandas datetime object to seconds
            ts.iloc[idx] = (end_time - row['timestamp']).total_seconds()

elapsed = time.time() - t0
print("Data processed in", elapsed, " seconds")

Add time left to dataframe `df`

In [ ]:
datta = pds.concat([df, ts], axis=1)
datta.head()

... and save it.

In [ ]:
datta.to_pickle('added_t.pkl')